In [1]:
%matplotlib inline
from os import listdir
from os.path import isfile, join
import numpy as np
from glob import glob
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib
import statsmodels.stats.multitest as smt
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from adjustText import adjust_text
from scipy import stats
from venn import venn
import itertools

In [15]:
def read_degenotate_mk(dir_path, file_name):
    
    ## Reads Degenotate MK table into a dict
    file_data = glob(dir_path + file_name)[0]
    df = pd.read_csv(file_data, sep='\t')
    df['standard_alpha'] = 1 - df['mk.odds.ni']
    mk_dict = df.set_index('gene').transpose().to_dict()
    return mk_dict

In [ ]:
def compute_standard_MKT(p, p0, d, d0):
    
    ## calculates standard alpha and DoS
    try:
        standard_alpha = 1 - p / p0 * d0 / d
    except ZeroDivisionError:
        standard_alpha = 'NA'
    try:
        standard_dos = d / (d + d0) - p / (p + p0)
    except ZeroDivisionError:
        standard_dos = 'NA'
    return standard_alpha, standard_dos


def compute_extended_MKT(p, p0, d, d0, p_high, p0_high):
    
    ## calculates extended alpha and DoS
    try:
        ext_alpha = 1 - p_high / p0_high * d0 / d
    except ZeroDivisionError:
        ext_alpha = 'NA'
    try:
        ext_dos = d / (d + d0) - p_high / (p_high + p0_high)
    except ZeroDivisionError:
        ext_dos = 'NA'
    return 


def compute_imputed_MKT():
    
    ##
    return 



In [18]:
# dbs = ['vidMac', 'vidCha', 'indInd', 'anoImb', 'molAte', 'agePho', 'picPub']
dbs = ['vidMac', 'vidCha', 'indInd', 'anoImb']
dir_path = '/Users/osipova/Documents/LabDocs/Brood_parasites_analysis/'

# fig, axes = plt.subplots(3, 3, figsize=(3*5, 3*5))
# p = sns.color_palette('Paired')

all_DFs = {}
i = 0

for db in dbs:
    ## Read data for each gene with syn/nonsyn AF
    print(db)
    file_name = 'MK_test_{}_ncbi/af.gene.longest.mk.tsv'.format(db)
    mk_dict = read_degenotate_mk(dir_path, file_name)
    
    all_DFs[db] = mk_dict
    
    
    

vidMac


/Users/osipova/local/Homebrew/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys


vidCha


ParserError: Error tokenizing data. C error: Expected 11 fields in line 4, saw 13


In [ ]:
### Imputed MKT
i = 0
cutoff = 0.15
delta = {}
p_wd_list = []
compare_alpha = {}
compare_dos = {}

with open('impMKT.dos.tsv', 'w') as ouf:
    for g in mk_dict:
        d  = mk_dict[g]['dN']
        d0 = mk_dict[g]['dS']
        pN_list = [float(i) for i in mk_dict[g]['af_nonsyn'].split(',')]
        pS_list = [float(i) for i in mk_dict[g]['af_syn'].split(',')]
        p  = len(pN_list)
        p0 = len(pS_list)
        
        dos = mk_dict[g]['dos']
        if dos != 'NA':
#             print(dos)
            i += 1


        ## calculate imputed alpha and DoS
        pN_af_high = [i for i in pN_list if i > cutoff]
        pN_af_low  = [i for i in pN_list if i < cutoff]
        pS_af_high = [i for i in pS_list if i > cutoff]
        pS_af_low  = [i for i in pS_list if i < cutoff]
        p_high  = len(pN_af_high)
        p0_high = len(pS_af_high)
        p_low   = len(pN_af_low)
        p0_low  = len(pS_af_low)

        try:
            p_wd = p_low - p_high / p0_high * p0_low
            imputed_alpha = 1 - (p - p_wd) / p0 * d0 / d
        except ZeroDivisionError:
            imputed_alpha = 'NaN'
        try:
            p_wd = p_low - p_high / p0_high * p0_low
            imputed_dos = d / (d + d0) - (p - p_wd) / ((p - p_wd) + p0)
        except ZeroDivisionError:
            imputed_dos = 'NaN'

        compare_alpha[g] = (standard_alpha, ext_alpha, imputed_alpha, p_wd, mk_dict[g]['pval'])
        compare_dos[g] = (standard_dos, ext_dos, imputed_dos, p_wd, mk_dict[g]['pval'])
#                 ouf.write('{}\t{}\t{}\n'.format(g, imputed_alpha, 'imputed'))
        ouf.write('{}\t{}\t{}\n'.format(g, imputed_dos, 'imputed'))
#                 delta[g] = (imputed_alpha - standard_alpha, imputed_alpha - ext_alpha)
        try:
            delta[g] = (imputed_dos - standard_dos, imputed_dos - ext_dos)
        except TypeError:
            pass
        p_wd_list.append(p_wd)

print(i)